In [1]:
import xarray as xr
import netCDF4
import numpy as np
import pandas as pd 
import pyeto as pt
import glob, os
import aasi

In [5]:
# User-supplied

mapfile = '/glade/p/work/manab/ff/forcinggeneration/mappingislandpark.nc'
forcdir = '/glade/p/work/manab/ff/islandpark/rawinput'
obsfile = '/glade/p/work/manab/ff/forcinggeneration/pilot_basin_data/isli_bor_flow_data.txt'
outdir = '/glade/p/work/manab/ff/islandpark/input/'
obsvar = 'dailydisc'

def calcWeightedAvg(ncdat, varname):
    '''
    Calculates the weighted average 
    of any <varname> in the mapping file
    '''
    wAvg = sum(ncdat[varname]*ncdat['weight'])
    return(wAvg)

def calcPET(lat, time, tmin, tmax, tmean):
    '''
    Calculates Potential Evapotranspiration using 
    Hargreaves equation (Hargreaves and Samani, 1985) 
    '''
    
    latrad = pt.deg2rad(lat)                             #Latitude to radians
    dayofyear = pd.Series(time).dt.day.values
    etrad = []
    pet = []
    
    # Calculate ET radiation
    for x in np.nditer(dayofyear):
        soldec = pt.sol_dec(x)                           #Solar declination
        sha = pt.sunset_hour_angle(latrad, soldec)       #Sunset hour aingle
        ird = pt.inv_rel_dist_earth_sun(x)               #Inverse relative distance Earth-Sun
        etrad.append(pt.et_rad(latrad, soldec, sha, ird))#Extraterrestrial radiation
    
    # Calculate PET using hargreaves
    for x in range(0, len(etrad)):
        pet.append(pt.hargreaves(tmin[x], tmax[x], tmean[x], etrad[x]))
    
    pet = np.array(pet)
    return(pet)

def impqobs(file, starttime, endtime):
    '''
    Import Flow observation for the period
    '''
    dat = pd.read_table(file, skiprows = 14, header = None, 
                    names = ['time', 'gaugeheight', 'resinflow', 'dailydisc', 'unregflow'])
    dat['time'] = pd.to_datetime(dat['time'])
    dat = dat.set_index(['time'])   #Set time as index
    dat = dat.loc[starttime:endtime]
    return(dat)
    
def ncextract(ncfile):
    '''
    Open each forcing netCDF file
    '''
    forcdat = xr.open_dataset(ncfile)
    
    # Extracts the variables values from forcing file
    prcp = forcdat['prcp'].values
    tmax = forcdat['tmax'].values
    tmin = forcdat['tmin'].values
    time = forcdat['time'].values
    tmean = (tmax+tmin)/2
    return(forcdat, prcp, time, tmin, tmax, tmean)

In [6]:
if __name__ == '__main__':
    
    # Mapping file extraction
    mapdat = xr.open_dataset(mapfile)
    lat = calcWeightedAvg(mapdat, 'latitude')
    lon = calcWeightedAvg(mapdat, 'longitude')
    
    # Processing forcing files
    forcfiles = glob.glob(forcdir + '/*.nc') #List of all files
    forcfiles.sort()
    
    for count, value in enumerate(forcfiles):
        [forcdat, prcp, time, tmin, tmax, tmean] = ncextract(value)
        
        pet = calcPET(lat, time, tmin, tmax, tmean)
        prcp = np.reshape(prcp,(prcp.shape[0],1,1))
        pet = np.reshape(pet,(pet.shape[0],1,1))
        tmean = np.reshape(tmax,(tmean.shape[0],1,1))

        #Extract observation for that period
        maxtime = forcdat['time'].max()
        mintime = forcdat['time'].min()
        obs = impqobs(obsfile, mintime, maxtime)
        obs = obs[obsvar].values
        obs = np.reshape(obs,(obs.shape[0],1,1))
        
        # Create new netCDF file
        f = xr.Dataset({'pr': (['time','latitude', 'longitude'],  prcp),
                        'pet': (['time','latitude', 'longitude'],  pet),
                        'temp': (['time','latitude', 'longitude'],  tmean),
                        'q_obs': (['time','latitude', 'longitude'],  obs)
                       },
                       coords={'time': time,
                               'latitude':[lat],
                               'longitude':[lon]}
                      )
        # f['q_obs'] = f['pr']*0+0.25

        f.to_netcdf(outdir+os.path.basename(value))  
        print("Creating FUSE forcing file " , count+1, " of ", len(forcfiles))

Creating FUSE forcing file  1  of  25
Creating FUSE forcing file  2  of  25
Creating FUSE forcing file  3  of  25
Creating FUSE forcing file  4  of  25
Creating FUSE forcing file  5  of  25
Creating FUSE forcing file  6  of  25
Creating FUSE forcing file  7  of  25
Creating FUSE forcing file  8  of  25
Creating FUSE forcing file  9  of  25
Creating FUSE forcing file  10  of  25
Creating FUSE forcing file  11  of  25
Creating FUSE forcing file  12  of  25
Creating FUSE forcing file  13  of  25
Creating FUSE forcing file  14  of  25
Creating FUSE forcing file  15  of  25
Creating FUSE forcing file  16  of  25
Creating FUSE forcing file  17  of  25
Creating FUSE forcing file  18  of  25
Creating FUSE forcing file  19  of  25
Creating FUSE forcing file  20  of  25
Creating FUSE forcing file  21  of  25
Creating FUSE forcing file  22  of  25
Creating FUSE forcing file  23  of  25
Creating FUSE forcing file  24  of  25
Creating FUSE forcing file  25  of  25
